# Parallelization Deep Dive

Parallelism is an absurdly messy topic, but we're doing to give it a shot in this notebook.

Parallelism is built on he simple observation: doing two things at a time, rather than just one, usually means your computer can get work done faster.  Consider the example of parsing a lot of JSON files.  If you have to parse one at a time, it might take you ten minutes.  But if you can find a way to parse two at a time, it will only take you ix minutes!  (Why not five minutes, you ask?  We'll talk about that later.  It's part of how parallelism gets very complicated).

Every consumer computer for the last fifteen or so years has a multi-core CPU.  This means the CPU has more than one physically independent piece of silicon in it that can work on it own set of tasks.  So in the JSON parsing example, we could tell our computer to just use one core to process JSONs (one at a time), or to use two cores (parse two at a time), or three cores, or four cores, and so on.  (_Note:_ I'm going to use the terms "cores" and "threads" pretty interchangeably.  They are not the same, but the differences aren't very important for us).  This JSON example is a specific kind of parallelism called an _embarassingly parallel_ workload: you have to do basically no work to separate it into different parallalizeable component.  Each JSON can be parsed completely independently, so the task of "parse some JSONs" very naturally lends itself to parallelization.

Parallelizing your code comes at the expense of code complexity, though for embarassingly parallel workloads, this extra complexity is usually minimal.  But it can add a lot of _conceptual_ complexity, including a lot of pitfalls that can sometimes make parallel code _slower_ than non-parallel code.

# Three kinds of concurrency

Parallelism is actually a special case of _concurrency._  Concurrency is just whenever your computer has more than one task ongoing at the same time, even if it's only working on one at a time.  There are three major kinds of concurrency: _asynchronous_ (or just _async_) programming,_ _multi-threading,_ and _multi-processing._  We're going to focus on multiprocessing.  Asynchronous programming and threading are very useful, but mostly when you're dealing with _I/O-bound_ workloads.  Multiprocessing is really the only way that Python can speed up _CPU-bound_ workloads.  For the sake of completeness, though, here's the quick overview.

_Asynchronous_ programs only ever do one thing at a time, but they might have multiple things pending at any given time.  The program will "juggle" which task is being worked on at any given moment.  This is like having homework for multiple classes: you have to get all your homework done, but you probably split the time up between your math problem set, your physics lab report, and your history essay.  At any given moment you're only doing one, but you are jumping between which one you work on in any given moment.

_Multithreaded_ programs allow your computer to actually work on more than one thing simultaneously.  Most languages use multithreading to speed up CPU-bound tasks, but as we'll discuss in a moment, the way Python is built prevents us from doing this.  Multi-threading is when you have one program (or, a bit more specifically: a single operating system-level _process_, e.g., something that would show up as one entry in Task Manager or htop or whatever Mac uses) doing multiple things at once.  All the threads share the same pool of RAM that the main process does, and can directly work on exactly the same data.

_Multiprocessing_ programs take multithreading one step farther.  Rathe than one process that does multiple things at a time, a multiprocessed program just spawns multiple completely independent processes.  Each process gets its own pool of RAM that it, and only it, has access to, and each process is treated as a separate running program by the operating system.  This has some ups and downs compared to threading.
- Cons:
    - Creating a process is much slower than creating a thread.  A whole process has to be set up that can run completely on its own--this just fundamentally requires more work than setting up a thread within an existing process.
    - Sending data between processes requires making a full copy of the data to be sent.  This can be slow, and can lead to multiprocessed program being _slower_ than single-processed ones.  There are often ways around this, but it does still add extra stuff you have to worry about.
- Pros:
    - Each process is much more independent, so one process crashing is less of an issue for the overall program.
    - Since each process is pretty independent and self-contained, they don't have to run on the same computer.  You could have multiple computers connected to one another, and send the different processes to different machine.  (This is the idea behind _cluster_ computing).

# Performance does not scale linearly with the number of threads/processes

## Diminishing Returns

Every single project--programming or otherwise--has some part that just can't be parallelized.  Furthermore, all forms of parallelization do add some fixed computaitonal overhead (e.g., every new thread might require 1mb of RAM and take 10ms to set up), and the more stuff you do in parallel, the less each parallel worker has to do.  This ends up meaning that every additional thread/process you add to your project has diminishing returns for the overall speed of the program.

This same thing is true for teams of people working on a single project.  One person trying to complete a large project is going to be slow.  Two people is going to be faster--but not exactly twice as fast, since now you need to set aside time to train the second person on what the project is, and the two people need some time to coordinate their work.  Same thing for hiring three, or four, or five people; each person requires training, and each person increases the amount of coordination that has to be done.  At some point, you'll reach the point where hiring another person actually slows the project down.

This is almost exactly how it works for threads and processes.  Every new thread/process requires some time to create and assign a task to.  Coordinating the different threads/processes, e.g. by combining their results or making sure they keep receiving new tasks, takes a bit of time.

Unfortuntely, the only way to really figure out where your program lies along this curve of diminishing returns is to experiment with the number of thread/processes and see if your performance gets better or worse.

## Over-subscription is death

_Over-subscription_ is when you ask your computer to do more things at a time than it physically can.  Just like with people, this leads to everything taking longer.  When you're parallelizing a CPU-bound workload, over-subscription will often make your code run slower than single-threaded code, and also often makes your computer grind to a screeching halt and become unresponsive.

**You need to know how many threads your computer has.**  This is easy to check on all operating systems, but each OS and each version need you to check it in different ways, so you'll need to spend a few minutes googling this on your machine.  My rule of thumb for CPU-bound tasks is to use, at most, the total number of threads your CPU has, minus two.  Leaving two threads free means your operating system still has resources available to do stuff it needs to do in the background.

That said: the absolute worst case scenario is that you create too many threads, your computer freezes up, and you have to do a hard reboot (holding down the power button for a few seconds).  This can result in lost work just like any unexpected shutdown, but it's not anything uniquely catastrophic.

# Python's problems with threading

CPython--the version of Python you're using, unless you went out of your way to download something like Jython or IronPython--has something called the _global interpreter lock_ (GIL) that stops more than one thread from executing code at the same time.

In programming, a _lock_ (or _mutex_, from **mut**ually **ex**clusive) is any object that's "passed around" between different threads to ensure that only one thread can do a particular thing at a time.  E.g.: you pretty much never want multiple threads to write to the same file at the same time.  So you'd have a lock associated with each file you're writing to, and whenever a thread needs to write data, it needs to _acquire_ this lock, and then it need to _release_ the lock once it's done, so that another thread can acquire it.  (Think of them like hall passes: there's only one hall pass per classroom, and if a student wants to leave the classroom, they need to acquire the pass, then return it when they get back, so another student can take it and leave).

Usually, you have multiple lock in your program, one for each thing that needs some sort of access control like this.  In CPython, there is a _global_ lock (the GIL).  Rather than being used to grant exclusive access to something like a file, the GIL grants permission to _run code._  So if you have multiple threads in your Python program, only one of them can run code at a time, because it needs to acquire the GIL before it is allowed to do anything.

This probably sounds like a weird choice.  That's because it is.  But, it is a choice that was made very deliberately in the early days of Python.  Back in the 90s (Python was first released in 1991), multi-core/multi-threaded computers were quite rare, so there wasn't a lot of demand or need for parallel computing support.  The GIL is designed to ensure that single-threaded programs run fast, and run without certain kinds of bugs (like two threads writing data to the same file at the same time).  It actually does a really good job at this.  But this comes at the cost of multi-threading performance.  Multi-threading in Python is not capable of speeding up CPU-bound workloads.  Multithreading is still an option, but we won't have much use for it.

So, we have to use multiprocessing.  Fortunately: Python makes this very, _very_ easy to get started with.  It will handle all the messy logic of creating and copying data between threads for us, which is a huge relief.

Note: some libraries, like Numpy and scikit-learn, are not actually written in Python.  They're often written in C, Fortran, or sometimes this cool language called Cython (not to be confused with CPython).  These libraries are capable of bypassing the GIL all together: they can write code in these other languages that makes use of thread-based parallelism for much more efficient speedups.  Unfortunately, if we want to do this ourselves, we have to leave the world of Python and start writing code in these other, much more difficult languages.  So we won't be doing that in this workshop.

# Multiprocessing in Python: Quickstart

**NOTE:  Jupyter does play well multiprocessing, especially on Windows.**  I can't actually run multiprocesing code in Jupyter cells directly (this is one of the reasons I think Jupyter is not a great development environment for production code).  Instead, I have to put the code in a separate .py file, import it in this notebook, and run functions from the imported file.  I'll embed the code from the files into the notebook, but they won't be in code cells.

Every time the `multiprocessing` library creates a new process, it uses a clever trick to copy over the global program state (i.e. copy over the imports, functions, classes, data, etc): each worker process imports the .py file that you hit "run" on.  This clever trick works startlingly well, but it has one _big_ caveat you need to be aware of:

**Any part of your code that spawns multiple processes _must_ be inside an `if __name__ == "__main__"` block.** Otherwise, it will get executed when the file is imported, which causes multiple processes to spawn, each of which imports the main file, executes the code that spawns threads, and so on and so forth.

## First example: parallelizing prime factorizations

Let's do a simple example: we'll write a very basic (and very dumb) primality test.  The test will take a positive integer `n`, and return `True` if `n` is prime, or `False` if `n` is composite.  This is the kind of code that Python is actually _super_ bad at--number-crunching work with really "tight" loops--so this is a great candidate for multiprocessing.  (Though accelerating it with Cython or Numba would generally be better--but we're focusing on multiprocessing right now).

```python
import multiprocessing

def is_prime(n):
    if n <= 2:
        return True
    else:
        for i in range(3, n // 2 + 1, 2):
            if n % i == 0:
                return True
    return False

def is_prime_parallel(up_to, chunksize=1, num_workers=2):
    """Parallelize the Collatz length calculation."""
    with multiprocessing.Pool(num_workers) as P:
        lengths = P.imap_unordered(
            is_prime, 
            list(range(1, up_to)),
            chunksize=chunksize
        )
        return list(lengths)
```

That's really all there is to it when you're running basic parallel code in Python.  Use a `multiprocessing.Pool(n_processes)` object and call some methods on it.  `.imap_unordered()` is probably the one you should generally use, for a few reason:

1. It is _lazy._  It will create an iterator that only consumes values and does work when you ask for the next value out of it.  Like all lazy iterators in Python, this means you can point it at something like a file or a database that's too large to fit into memory.  As long as you're saving your result to a file/database/etc as you get them, you can use this to process enormous amounts of data with a pretty small memory footprint.
2. It is _non-deterministically ordered._  I.e.: if you ask it to parallelize over the values [1, 2, 3], you might get the results back for [3, 1, 2].  This rarely actually matters in data processing; we almost always multiprocess over things where their exact order doesn't matter.  This also has a big performance benefit: using other methods like `.map()` or `.imap()` will actually make the process pool pause and wait for things to finish in the right order, so it can return things in the correct order.  This can sometimes have a significant effect.

There are a few other methods on `multiprocessing.Pool()` objects, but I've personally never found them as useful a `.imap_unordered()`.

We would hope that running the parallel version is about twice as fast as just iterating through numbers in a single thread/process and calculating the lengths there.  Let's test this.

In [1]:
import parallel_primes
        
if __name__ == "__main__":
    %time sum([parallel_primes.is_prime(i) for i in range(1, 25_000)])
    %time parallel_primes.is_prime_parallel(25_000, chunksize=1, num_workers=4)

CPU times: total: 922 ms
Wall time: 929 ms
CPU times: total: 2.97 s
Wall time: 2.9 s


Huh.  That's not what we had hoped for.  Multiprocessing was supposed to speed our code up, but it made it slower!

There's one major issue at play here: _serialization overhead._  Essentially, every time we need to do something to one of the numbers, we need to copy it to the worker process.  As it turns out, it's actually very, very slow to do a lot of small copy operations; it's _hugely_ faster to do one large copy operation, if we can.  (It's not too dissimilar from shipping physical items to someone you know: would you rather put every single thing in its own box, or try to put more things into fewer, but larger, boxes?).  But also, in the above code, for a lot of smaller numbers, checking if they're prime is extremely fast.  Faster than the time it takes to copy the number to the worker process and to copy the result back to the parent process.  This is why the single-threaded code is faster in this instance: it's doing the same calculations, but it doesn't have to deal with any of the extra work of copying values between different priocesses.

So how do we deal with this?  Easy: _chunking_.  We can tell Python to send, say, 1,000 numbers at a time, rather than just 1.  The process receiving the chunk of values will deal with iterating through it and returning things correctly; we don't need to do anything too special.  All we do is add the `chunksize=` argument to the `.imap_unordered()` method.

In [2]:
import parallel_primes
        
if __name__ == "__main__":
    %time sum([parallel_primes.is_prime(i) for i in range(1, 25_000)])
    %time parallel_primes.is_prime_parallel(25_000, chunksize=5000, num_workers=4)

CPU times: total: 875 ms
Wall time: 877 ms
CPU times: total: 31.2 ms
Wall time: 533 ms


Much better!

This is a pretty extreme example, I admit; most thing you do will probably give you a good speedup without too much chunk size tuning.  That said, here are some good rules of thumb:

- Offload as much work as possible to the worker processes (i.e., do as much as you can in the functions that you're passing to `.imap_unordered()`).
- Pass as little data as possible between processes to minimize serialization overhead.
- Pass the _simplest_ data possible between processes.  This means any of Python's non-container types: ints, floats, strings, etc.  (more complex objects like lists and dictionaries get serialized using the `pickle` module, and de-serialized on the other end; this can add quite a lot of overhead for larger and more complex objects).
- If you're not able to put a huge amount of work into the worker process, increase your chunksize.  You'll have to play with it a bit to find a good number; it's really hard to reason about a good value ahead of time, unfortunately.
- If you can't get multiprocessing to work, maybe your code should just be single threaded.

# Easy embarassing parallelism with `joblib`

Python's `multiprocessing` library is great--it's flexible, it's in the standard library, and it's got a pretty convenient API.  (Almost all of the complexity comes from the fact that parallelism, no matter what language or tool you're in, is just fundamentally complicated).  But, there's another library that adds some nice extra tools: `joblib`.  Most of the advantages of `joblib` are pretty technical: things like allowing shared memory views of Numpy `array`s, rather than copying them between all the child processes, and letting you specify that the code you're writing will bypass the GIL, and offering some extra backends for the parallelism.

`joblib` also provides a slightly different API for multiprocessing that is sometimes nicer than the built-in library, but can have a few drawbacks.  Namely, it's more oriented towards _eager_ evaluation: it will try to consume the entirety of the iterable you pass it, and will churn away until it's finished processing everything you've given it.  This is rarely an issue in practice, but it does add a little extra work if you need both the lazier behavior and some of the extra features of `joblib`.

The basic sketch for `joblib` code goes like this:

In [3]:
%%time
from joblib import Parallel, delayed
import parallel_primes

if __name__ == "__main__":
    parallel_dispatcher = Parallel(n_jobs=4)
    parallel_func = delayed(parallel_primes.is_prime)
    res = parallel_dispatcher(
        parallel_func(i)
        for i in range(1, 25_000)
    )
    print(res[:10])
    

[True, True, False, False, False, True, False, False, True, True]
CPU times: total: 375 ms
Wall time: 1.51 s


Breaking this down:

- `joblib.Parallel(n_jobs)` creates an object that manages the parallelism logic for you.  This object is _callable_: it behaves like a function.  When you need to use it to parallelize some work, you pass it an iterable of `delayed()` objects (see the next bullet point below).  The `Parallel()` object then handles dispatching these jobs to worker processes, and returns a list with each worker's result.
- `joblib.delayed()` is essentially a "wrapper" around a function.  You pass it some function, and it returns a "wrapped" version of that function that just does delayed execution.  So: `joblib.delayed(parallel_primes.is_prime)(i)` will more or less tell Python "don't execute `parallel_primes.is_prime(i)` right now.  Hold off on that until I tell you to."  `delayed()` also does some stuff to make the function more friendly to parallelization.  It basically packages up the work to be done in a way that `Parallel()` knows how to send off to individual workers.

As mentioned, though, `joblib` likes _eager evaluation._  As long as your data fits in RAM and has a reasonable amount of wiggle room, this is fine, and you might even find `joblib` runs a bit faster than `multiprocessing`.

Here's my general rule of thumb for which one to use:
- Either one is great for general-purpose parallelism.  Just pick one to be your default and don't worry about it.
- `multiprocessing` makes it easier to write code that deals with each result as it is calculated.  It's easier to use `multiprocessing` to deal with data that's too big to fit in RAM.
- `joblib` is generally more flexible, but making use of that extra flexibility requires learning a _lot_ more about parallism than we can cover here.  `joblib` tends to be better if the data you need to send between processes is a Numpy `array` or a few similar data strucures; it can avoid some of the data copy overhead.  `joblib` _can_ be used for lazy evaluation, but you'll have to do a bit more work yourself to manage the iteration.

# Deeper Dive: Python's default serialization

_Serialization_ is a term we've seen a few times before.  As a reminder: serialization means converting a code object into a linear, one-dimensional representation that is _memory-contiguous._  Serializing objects is required to save them to files (files are just sequential 1s and 0s on your hard drive), and to send them between processes.

Some examples of serialization:

- Representing a Pandas `DataFrame` as a CSV string.  The indvidual columns of the `DataFrame` are memory-contiguous, but the `DataFrame` as a whole may not be.
- Converting a `dict` to a JSON-formatted string.  The `dict`'s elements are not guaranteed to be next to each other in memory.
- Saving a scikit-learn model to file.  This requires collecting all the model's bits and pieces into one place to save.

Python's default serialization tool for multiprocessing is the `pickle` module of the standard library.  `pickle` has its pros and cons.  It's extremely general: just about anything that's a Python object can be serialized using `pickle` (with a very small number of exceptions, like `lambda` functions). But, this generality comes at a performance cost for some objects.  Usually `pickle` is pretty good though: it serialized the _in-memory representation_ of the object; unlike JSON serialization, the output is raw binary data, not human-readable strings, which can save some time.

To demonstrate this time-saving: let's do a quick benchmark.  We'll create a huge, ugly, super deeply-nested dictionary, and then serialize it using `pickle` and `orjson`--`orjson` is currently the fastest JSON serializer out there.

In [4]:
import random
import string

def random_list(length):
    """Generate a list of random integers, but each entry
    has a chance to instead be a different nested data structure."""
    _list = []
    for i in range(length):
        # 5% chance to be a dict or list.
        _rand = random.random()
        if _rand >= 0.5:
            _list.append(random.randint(0, 1_000_000_000))
        elif _rand >= 0.05:
            _list.append(random_string(random.randint(100, 300)))
        else:
            _list.append(random.choice([random_list, random_dict])(random.randint(5, 25)))
    return _list

def random_dict(length):
    """Generate a random dictionary with `length` keys.
    Each value is a string, number, dict, or list."""
    _dict = {}
    for i in map(str, range(length)):
        _rand = random.random()
        if _rand >= 0.5:
            _dict[i] =  random.randint(0, 1_000_000_000)
        elif _rand >= 0.05:
            _dict[i] = random_string(random.randint(100, 300))
        else:
            _dict[i] = random.choice([random_list, random_dict])(random.randint(5, 25))
    return _dict
            
def random_string(length):
    """Generate some random text."""
    return "".join(random.choices(string.printable, k=length))

test_dict = random_dict(500)

In [5]:
import pickle
from timeit import timeit

pickle_time = timeit("pickle.dumps(test_dict)", globals=globals(), number=10000)
print(f"Pickle serialization, 10,000 runs: {pickle_time:.2f}s")

Pickle serialization, 10,000 runs: 3.00s


In [6]:
from timeit import timeit

import orjson

orjson_time = timeit("orjson.dumps(test_dict)", globals=globals(), number=10000)
print(f"ORJSON serialization, 10,000 runs: {orjson_time:.2f}s")

ORJSON serialization, 10,000 runs: 5.83s


However: `orjson` is designed to generate human-readable data.  `pickle` is designed to generate machine-readable data.  So they aren't really designed for the same purposes, and comparing them like this isn't necessarily fair.

In [7]:
test_dict = {
    "Numbers": [1,2,3,4,5],
    # for orjson compatibility, dict keys must alwys be strings
    "Squares": {"1": 1, "2": 3, "3": 9, "4": 16, "5": 25}
}

print("Pickle representation:")
print(pickle.dumps(test_dict))

print("\norjon representation:")
print(orjson.dumps(test_dict))

Pickle representation:
b'\x80\x04\x95I\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x07Numbers\x94]\x94(K\x01K\x02K\x03K\x04K\x05e\x8c\x07Squares\x94}\x94(\x8c\x011\x94K\x01\x8c\x012\x94K\x03\x8c\x013\x94K\t\x8c\x014\x94K\x10\x8c\x015\x94K\x19uu.'

orjon representation:
b'{"Numbers":[1,2,3,4,5],"Squares":{"1":1,"2":3,"3":9,"4":16,"5":25}}'
